# Run a massive number of simulations

## Instructions

1. Set parameter set to simulate in simulation_settings.yml
2. Set output file path in this workbook
3. Run this workbook

Alterantively, you can also run the simulation from the command line:

```
python process_simulation.py --settings simulation_settings.yml --output local_results/simulation_results.csv
```

In [ ]:
# imports
import process_simulation
import subprocess
import sys

# see simulation_settings.yml for configuration of simulation settings
settings_file = 'simulation_settings.yml'

# set the output files for running the experiments
output_file = 'local_results/simulation_results.csv'
output_file_sensitivity_analysis = 'local_results/sensitivity_analysis_results.csv'

##  Run the experiments

In [ ]:
cmd = ["python", "-u", "process_simulation.py", "--settings", settings_file, "--output", output_file, "--random_order"]

# Start the process and capture its output in real time
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

while True:
    line = process.stdout.readline()
    if line:
        sys.stdout.write(line)
        sys.stdout.flush()
    elif process.poll() is not None:
        break

## Run the sensitivity analysis

In [ ]:
cmd = ["python", "-u", "process_simulation.py", "--settings", settings_file, "--output", output_file_sensitivity_analysis, "--random_order", "--sensitivity_analysis"]

# Start the process and capture its output in real time
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

while True:
    line = process.stdout.readline()
    if line:
        sys.stdout.write(line)
        sys.stdout.flush()
    elif process.poll() is not None:
        break

## Performance analytics

In [ ]:
# code for performance analytics
# set configuration for non-automated process, no exceptions
t = 5000 # number of iterations (5000 default)
l = 100 # lexicon: number of different process steps
m = 5 # number of subunits of the process, needs to be a factor of l
r = 50 # number of historic sequences remembered
n = 5

# set variability metrics
v_m = 0.005
v_a = 0
v_m_e = 0.01
v_a_e = 0.03

# set automation degree and exception percentage 
a = 0
e = 0

# set the seed to an integer value for reproducibility
seed = 1

# create simulation
my_simulation = process_simulation.ProcessSimulationModel(t, l, m, r, n, v_m=v_m, v_a=v_a, v_m_e=v_m_e, v_a_e=v_a_e, a=a, e=e, seed=seed)

# run the simulation and store runtime dynamics
import cProfile
import pstats
from io import StringIO
import os

# Create a profiler object
profiler = cProfile.Profile()

# Start profiling
profiler.enable()

# Run the simulation
adjacency_matrices = my_simulation.run_simulation(normalize_adjacency_matrices=True)

# Stop profiling
profiler.disable()

# Create a stream to hold the profile data
stream = StringIO()

# Create a Stats object and sort the profile data
stats = pstats.Stats(profiler, stream=stream).sort_stats(pstats.SortKey.CUMULATIVE)

# Print the profile to stream
stats.print_stats()

# Display the profile data in the Jupyter notebook
profile_data = stream.getvalue()
print(profile_data)

# Optionally, you can save the profile data to a file for later analysis
# Create folder if it does not exist
if not os.path.exists("local_results"):
    os.makedirs("local_results")

with open("local_results/profile_output.txt", "w") as f:
    f.write(stream.getvalue())